In [1]:
# Подавление предупреждений
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

# Импорт необходимых библиотек
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel,AutoModelForMaskedLM
import torch
import torch.nn.functional as F
from torch import Tensor
from einops import rearrange
from typing import Tuple, Callable
from torch.autograd import Function
import gc
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Данные

In [3]:
from torch.utils.data import Dataset, DataLoader 
import numpy as np 
import math 

class Dataset_MELD_RESD(): 
    def __init__(self, part='train', transform=None): 
        if part == 'train':
            df_meld = pd.read_csv("train_sent_emo.csv")[['Utterance', 'Emotion']]
            df_meld.columns = ['text', 'emotion']
            df_resd = pd.read_csv("train.csv")[['text', 'emotion']]
            df = pd.concat([df_meld, df_resd[0:int(len(df_resd)*0.7)]], axis=0)
        elif part == 'dev_meld':
            df = pd.read_csv("dev_sent_emo.csv")[['Utterance', 'Emotion']]
            df.columns = ['text', 'emotion']
        elif part == 'dev_resd':
            df = pd.read_csv("train.csv")
            df = df[int(len(df)*0.7):]
        elif part == 'test_resd':
            df = pd.read_csv("test.csv")
        elif part == 'test_meld':
            df = pd.read_csv("test_sent_emo.csv")[['Utterance', 'Emotion']]
            df.columns = ['text', 'emotion']
        elif part == 'test_resd':
            df = pd.read_csv("test.csv")
        else:
            raise ValueError('Unknown part of Dataset (train / test_meld / test_resd)')
        self.x = list(df['text'].values)
        emotion_mapping = {
            'anger': 0,
            'disgust': 1,
            'fear': 2,
            'joy': 3,
            'happiness': 3,
            'neutral': 4,
            'sadness': 5,
            'surprise': 6,
            'enthusiasm': 6
        }

        self.y = torch.tensor(df['emotion'].apply(lambda x : emotion_mapping[x]).values).to(device)
        self.n_samples = df.shape[0]

    def __getitem__(self, index): 
        return self.x[index], self.y[index] 
        
    def __len__(self): 
        return self.n_samples 

In [4]:
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset=Dataset_MELD_RESD('train'), batch_size=BATCH_SIZE, shuffle=True)
dev_meld_dataloader = DataLoader(dataset=Dataset_MELD_RESD('dev_meld'), batch_size=BATCH_SIZE, shuffle=False)
dev_resd_dataloader = DataLoader(dataset=Dataset_MELD_RESD('dev_resd'), batch_size=BATCH_SIZE, shuffle=False)
test_meld_dataloader = DataLoader(dataset=Dataset_MELD_RESD('test_meld'), batch_size=BATCH_SIZE, shuffle=False)
test_resd_dataloader = DataLoader(dataset=Dataset_MELD_RESD('test_resd'), batch_size=BATCH_SIZE, shuffle=False)

### Feature Extractor

In [5]:
class Embedding():
    def __init__(self, model_name='jina', pooling=None):
        self.model_name = model_name
        self.pooling = pooling
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if model_name == 'jina':
            self.tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v3", code_revision='da863dd04a4e5dce6814c6625adfba87b83838aa', trust_remote_code=True)
            self.model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", code_revision='da863dd04a4e5dce6814c6625adfba87b83838aa', trust_remote_code=True).to(self.device)
        elif model_name == 'xlm-roberta-base':
            self.tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
            self.model = AutoModel.from_pretrained('xlm-roberta-base').to(self.device)
        elif model_name == 'canine-c':
            self.tokenizer = AutoTokenizer.from_pretrained('google/canine-c')
            self.model = AutoModel.from_pretrained('google/canine-c').to(self.device)
        else:
            raise ValueError('Unknown name of Embedding')
    def _mean_pooling(self, X):
        def mean_pooling(model_output, attention_mask):
            token_embeddings = model_output[0]
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        encoded_input = self.tokenizer(X, padding=True, truncation=True, return_tensors='pt').to(self.device)
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.unsqueeze(1)
    
    def get_embeddings(self, X):
        if self.pooling is None:
            if self.model_name == 'canine-c_emb':
                max_len = 329
            else:
                max_len = 95
            encoded_input = self.tokenizer(X, padding=True, truncation=True, return_tensors='pt').to(self.device)
            with torch.no_grad():
                features = self.model(**encoded_input)[0].detach().cpu().float().numpy()
            res = np.pad(features[:, :max_len, :], ((0, 0), (0, max(0, max_len - features.shape[1])), (0, 0)), "constant")
            return torch.tensor(res)
        elif self.pooling == 'mean':
            return self._mean_pooling(X)
        else:
            raise ValueError('Unknown type of pooling')

### Метрики

In [6]:
def evaluate_metrics(model, test_dataloader):
    model.eval()
    y_test = []
    y_predict = []
    with torch.no_grad():
        for batch, (batch_X, targets) in enumerate(test_dataloader, 1):
            y_test.extend(list(map(int, targets)))
            output = model(batch_X)
            _, predictions = torch.max(output, dim=1)
            y_predict.extend(list(map(int, predictions)))
        # Unweighted Average Recall (UAR)
        uar = recall_score(y_test, y_predict, average='macro')
        # Weighted Average Recall (WAR)
        war = recall_score(y_test, y_predict, average='weighted')
        # Macro F1-score (MF1)
        mf1 = f1_score(y_test, y_predict, average='macro')
        # Weighted F1-score (WF1)
        wf1 = f1_score(y_test, y_predict, average='weighted')
    return {'uar': 100.0 * uar, 'war': 100.0 * war, 'mf1': 100.0 * mf1, 'wf1': 100.0 * wf1}

# Обучение

In [7]:
from dataclasses import dataclass
from typing import ClassVar
from typing import List, Dict, Any, Tuple, Optional
@dataclass
class ModelTrainer:
    model: 'typing.Any'
    train_dataloader: DataLoader
    dev_meld_dataloader: DataLoader
    dev_resd_dataloader: DataLoader
    test_meld_dataloader: DataLoader
    test_resd_dataloader: DataLoader
    device: torch.device
    epochs: int
    round_loss: int
    round_acc: int

    optimizer: torch.optim
    loss_fn: 'typing.Any'
    
    patience: int = 10 # Ранняя остановка обучения

    class_names: ClassVar[Optional[List[str]]] = None # Список имен классов

    def __post_init__(self):
        
        # История обучения и тестирования
        self.__history = pd.DataFrame({
            "train_avg": [], # Средние метрики на тренировочной выборке
            "dev_avg": [], # Средние метрики на валидационной выборке
            "train_loss": [], # Loss на тренировочной выборке
            "dev_loss": [], # Loss на валидационной выборке
        })

        # Количество шагов в одной эпохе
        self.__train_steps = len(self.train_dataloader)
        self.__dev_steps = len(self.dev_meld_dataloader) + len(self.dev_resd_dataloader)
        self.__test_steps = len(self.test_meld_dataloader) + len(self.test_resd_dataloader)

        self.__best_dev_avg = 0
        self.__no_improvement_count = 0
        
        self.loss_fn = self.loss_fn

    @property
    def history(self) -> pd.DataFrame:
        """Получение DataFrame историей обучения и тестирования

        Returns:
            pd.DataFrame: **DataFrame** c историей обучения и тестирования
        """

        return self.__history

    @classmethod
    def get_model_logits(cls, logits: torch.Tensor) -> torch.Tensor:
        """Получение логитов модели в зависимости от функции потерь

        Args:
            logits (torch.Tensor): Входные логиты

        Returns:
            torch.Tensor: Обработанные логиты
        """

        if isinstance(cls.loss_fn, nn.NLLLoss):
            log_softmax = nn.LogSoftmax(dim = 1)
            return log_softmax(logits)
        elif isinstance(cls.loss_fn, nn.CrossEntropyLoss):
            return logits

    def _is_best_model(self, dev_avg: float) -> bool:
        """Проверка, является ли текущая модель лучшей на основе метрик валидации

        Args:
            test_accuracy (float): Текущая точность тестирования

        Returns:
            bool: True, если текущая модель лучшая, иначе False
        """

        try:
            max_dev_avg = max(self.__history["dev_avg"])
        except ValueError:
            max_dev_avg = 0
        return dev_avg > max_dev_avg

    def _save_model(self, epoch: int, path_to_model: str, test_accuracy: float, loss: torch.Tensor) -> None:
        """Сохранение модели

        Args:
            epoch (int): Текущая эпоха
            path_to_model (str): Путь для сохранения модели
            test_accuracy (float): Точность на тестовой выборке
            loss (torch.Tensor): Значение потерь
        """
        
        os.makedirs(path_to_model, exist_ok = True)
        self._best_model_name = f"{self.model.__class__.__name__}_{self.model.model_name}_{epoch}_{test_accuracy}_checkpoint.pth"

        torch.save({
            "epoch": epoch,
            "model_state_dict": self.model.state_dict(),
            "optimizer_state_dict": self.optimizer.state_dict(),
            "test_loss": loss,
        }, os.path.join(path_to_model, f"{self.model.__class__.__name__}_{self.model.model_name}_{epoch}_{test_accuracy}_checkpoint.pth"))
    
    # Процесс обучения
    def train(self, path_to_model: str) -> None:
        """Процесс обучения

        Args:
            path_to_model (str): Путь для сохранения моделей

        Returns:
            None
        """
        
        losses_train_list = []
        losses_dev_list = []
        accuracy_train_list = []
        accuracy_dev_list = []

        for epoch in range(1, self.epochs + 1):
            with torch.no_grad():
                torch.cuda.empty_cache()
            self.model.train() # Установка модели в режим обучения
            # Сумма Loss
            total_train_loss = 0
            total_dev_loss = 0
            total_dev_loss_meld = 0
            total_dev_loss_resd = 0
            # Сумма точности
            train_accuracy = 0
            dev_accuracy = 0
            dev_accuracy_meld = 0
            dev_accuracy_resd = 0
            # Сумма метрик
            train_uar = 0
            train_war = 0
            train_mf1 = 0
            train_wf1 = 0
            dev_uar_meld = 0
            dev_war_meld = 0
            dev_mf1_meld = 0
            dev_wf1_meld = 0
            dev_uar_resd = 0
            dev_war_resd = 0
            dev_mf1_resd = 0
            dev_wf1_resd = 0

            # Проход по всем тренировочным пакетам
            with tqdm(total = self.__train_steps, desc = f"Эпоха {epoch}", unit = "batch") as pbar_train:
                for batch, (batch_X, targets) in enumerate(self.train_dataloader, 1):
                    targets = targets.to(device)
                    logits = self.model(batch_X)
                    loss = self.loss_fn(logits, targets) # Ошибка предсказаний

                    # Обратное распространение для обновления весов
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
        
                    total_train_loss += loss.item() # Потеря
                    # Метрики
                    train_uar += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                    train_war += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
                    train_mf1 += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                    train_wf1 += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
                    train_accuracy += (logits.argmax(1) == targets).type(torch.float).sum().item()
        
                    pbar_train.update(1)
                    with torch.no_grad():
                        torch.cuda.empty_cache()

                # Средняя потеря
                avg_train_loss = round(total_train_loss / batch, self.round_loss)
                losses_train_list.append(avg_train_loss)
        
                # Точность
                train_accuracy = round(train_accuracy / len(self.train_dataloader.dataset) * 100, self.round_acc)
                
                
                train_uar = round(train_uar / len(self.train_dataloader), self.round_acc)
                train_war = round(train_war / len(self.train_dataloader), self.round_acc)
                train_mf1 = round(train_mf1 / len(self.train_dataloader), self.round_acc)
                train_wf1 = round(train_wf1 / len(self.train_dataloader), self.round_acc)
                
                train_avg_metrics = 0.25 * (train_uar + train_war + train_mf1 + train_wf1)
                accuracy_train_list.append(train_avg_metrics)
        
                pbar_train.set_postfix({
                    "uar": train_uar,
                    "war" : train_war,
                    "mf1" : train_mf1,
                    "wf1" : train_wf1,
                    "avg" : train_avg_metrics,
                    "Средняя потеря": avg_train_loss
                })
            
            
            # Установка модели в режим предсказаний
            self.model.eval()
        
            # Предсказания на валидационной выборке
            with torch.no_grad():
                with tqdm(total = self.__dev_steps, desc = f"Тестирование {epoch}", unit = "batch") as pbar_dev:
                    num_batches = 0
                    for batch, (batch_X, targets) in enumerate(self.dev_meld_dataloader, 1):
                        targets = targets.to(device)
                        logits = self.model(batch_X)
                        loss = self.loss_fn(logits, targets) # Ошибка предсказаний
                        
                        total_dev_loss += loss.item() # Потеря
                        total_dev_loss_meld += loss.item()
                        dev_accuracy_meld += (logits.argmax(1) == targets).type(torch.float).sum().item()
                        # Метрики
                        dev_uar_meld += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                        dev_war_meld += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
                        dev_mf1_meld += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                        dev_wf1_meld += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
        
                        pbar_dev.update(1)
                        with torch.no_grad():
                            torch.cuda.empty_cache()
                    num_batches += batch
                    batch_meld = batch
                    for batch, (batch_X, targets) in enumerate(self.dev_resd_dataloader, 1):
                        targets = targets.to(device)
                        logits = self.model(batch_X)
                        loss = self.loss_fn(logits, targets) # Ошибка предсказаний
                        
                        total_dev_loss += loss.item() # Потеря
                        total_dev_loss_resd += loss.item()
                        # Количество правильных предсказаний
                        dev_accuracy_resd += (logits.argmax(1) == targets).type(torch.float).sum().item()
                        # Метрики
                        dev_uar_resd += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                        dev_war_resd += 100.0 * recall_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
                        dev_mf1_resd += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='macro')
                        dev_wf1_resd += 100.0 * f1_score(targets.cpu(), logits.argmax(1).cpu(), average='weighted')
        
                        pbar_dev.update(1)
                        with torch.no_grad():
                            torch.cuda.empty_cache()
                    num_batches += batch
                    # Средняя потеря
                    avg_dev_loss = round(total_dev_loss / num_batches, self.round_loss)
                    avg_dev_loss = round(0.5 * (total_dev_loss_meld / batch_meld + total_dev_loss_resd / batch), self.round_loss)
                    losses_dev_list.append(avg_dev_loss)
        
                    # Точность
                    dev_accuracy = round(0.5 * (dev_accuracy_meld / len(self.dev_meld_dataloader.dataset) * 100 + dev_accuracy_resd / len(self.dev_resd_dataloader.dataset) * 100), self.round_acc)
                
                    dev_uar_meld = round(dev_uar_meld / len(self.dev_meld_dataloader), self.round_acc)
                    dev_war_meld = round(dev_war_meld / len(self.dev_meld_dataloader), self.round_acc)
                    dev_mf1_meld = round(dev_mf1_meld / len(self.dev_meld_dataloader), self.round_acc)
                    dev_wf1_meld = round(dev_wf1_meld / len(self.dev_meld_dataloader), self.round_acc)
                    
                    dev_uar_resd = round(dev_uar_resd / len(self.dev_resd_dataloader), self.round_acc)
                    dev_war_resd = round(dev_war_resd / len(self.dev_resd_dataloader), self.round_acc)
                    dev_mf1_resd = round(dev_mf1_resd / len(self.dev_resd_dataloader), self.round_acc)
                    dev_wf1_resd = round(dev_wf1_resd / len(self.dev_resd_dataloader), self.round_acc)
                    
                    
                    dev_uar = 0.5 * (dev_uar_meld + dev_uar_resd)
                    dev_war = 0.5 * (dev_war_meld + dev_war_resd)
                    dev_mf1 = 0.5 * (dev_mf1_meld + dev_mf1_resd)
                    dev_wf1 = 0.5 * (dev_wf1_meld + dev_wf1_resd)
                    
                    dev_avg_metrics = 0.25 * (dev_uar + dev_war + dev_mf1 + dev_wf1)
                    accuracy_dev_list.append(dev_avg_metrics)
                    
                    pbar_dev.set_postfix({
                        "uar": dev_uar,
                        "war" : dev_war,
                        "mf1" : dev_mf1,
                        "wf1" : dev_wf1,
                        "avg" : dev_avg_metrics,
                        "Средняя потеря": avg_dev_loss
                    })
            
            if self._is_best_model(dev_avg_metrics):
                self._save_model(epoch, path_to_model, round(dev_avg_metrics, self.round_acc), avg_dev_loss)
                self.__best_dev_avg = dev_avg_metrics
                self.__no_improvement_count = 0
            else:
                self.__no_improvement_count += 1

            # Добавлениие данных в историю обучения
            new_row = pd.Series([train_avg_metrics, dev_avg_metrics, avg_train_loss, avg_dev_loss], index = self.__history.columns)
            self.__history = pd.concat([self.__history, new_row.to_frame().T], ignore_index = True)

            if self.__no_improvement_count >= self.patience:
                print(f"Ранняя остановка на эпохе {epoch} из-за отсутствия улучшения точности на тестовой выборке")
                break
        '''checkpoint = torch.load(os.path.join(path_to_model, self._best_model_name))
        self.model.load_state_dict(checkpoint['model_state_dict'])
        metrics_dev_meld = evaluate_metrics(self.model, dev_meld_dataloader)
        metrics_dev_resd = evaluate_metrics(self.model, dev_resd_dataloader)
        print("Метрики на валидационной выборке MELD: ", metrics_dev_meld)
        print("Метрики на валидационной выборке RESD: ", metrics_dev_resd)
        metrics_test_meld = evaluate_metrics(self.model, test_meld_dataloader)
        metrics_test_resd = evaluate_metrics(self.model, test_resd_dataloader)
        print("Метрики на тестовой выборке MELD: ", metrics_test_meld)
        print("Метрики на тестовой выборке RESD: ", metrics_test_resd)'''
        '''# Визуализация графиков потерь и точности
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.plot(losses_train_list, label = 'Потери на тренировочной выборке')
        plt.plot(losses_dev_list, label = 'Потери на валидационной выборке')
        plt.title('Потери во время обучения')
        plt.xlabel('Эпоха')
        plt.ylabel('Потери')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(accuracy_train_list, label = 'Средние метрики на тренировочной выборке')
        plt.plot(accuracy_dev_list, label = 'Средние метрики на валидационной выборке')
        plt.title('Средние метрики во время обучения')
        plt.xlabel('Эпоха')
        plt.ylabel('Точность')
        plt.legend()

        plt.tight_layout()
        plt.show()'''

    # Получение хэш-значения
    def __hash__(self):
        return id(self)

In [8]:
EPOCHS = 50 # Количество эпох
BATCH_SIZE = 32 # Размер выборки (пакета)
LEARNING_RATE = 1e-4 # Скорость обучения
ROUND_ACC = 2 # Знаков Accuracy после запятой
ROUND_LOSS = 7 # Знаков Loss после запятой
ROOT_DIR = os.path.join(".")
PATH_TO_MODEL = os.path.join(ROOT_DIR, "Models_mamba")

In [9]:
from sklearn.utils.class_weight import compute_class_weight
y = []
for batch, (batch_X, targets) in enumerate(train_dataloader, 1):
    y.extend(list(map(int, targets)))
class_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y), dtype=torch.float).to(device)

In [10]:
from torch.nn.functional import silu
from torch.nn.functional import softplus
from einops import rearrange, repeat, einsum
class RMSNorm(nn.Module):
    def __init__(self, d_model: int, eps: float = 1e-8) -> None:
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(d_model))

    def forward(self, x: Tensor) -> Tensor:        
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim = True) + self.eps) * self.weight

class Mamba(nn.Module):
    def __init__(self, num_layers, d_input, d_model, d_state=16, d_discr=None, ker_size=4, num_classes=7, model_name='jina', pooling=None):
        super().__init__()
        mamba_par = {
            'd_input' : d_input,
            'd_model' : d_model,
            'd_state' : d_state,
            'd_discr' : d_discr,
            'ker_size': ker_size
        }
        self.model_name = model_name
        embed = Embedding(model_name, pooling)
        self.embedding = embed.get_embeddings
        self.layers = nn.ModuleList([nn.ModuleList([MambaBlock(**mamba_par), RMSNorm(d_input)]) for _ in range(num_layers)])
        self.fc_out = nn.Linear(d_input, num_classes)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def forward(self, seq, cache=None):
        seq = torch.tensor(self.embedding(seq)).to(self.device)
        for mamba, norm in self.layers:
            out, cache = mamba(norm(seq), cache)
            seq = out + seq
        return self.fc_out(seq.mean(dim = 1))
        
class MambaBlock(nn.Module):
    def __init__(self, d_input, d_model, d_state=16, d_discr=None, ker_size=4):
        super().__init__()
        d_discr = d_discr if d_discr is not None else d_model // 16
        self.in_proj  = nn.Linear(d_input, 2 * d_model, bias=False)
        self.out_proj = nn.Linear(d_model, d_input, bias=False)
        self.s_B = nn.Linear(d_model, d_state, bias=False)
        self.s_C = nn.Linear(d_model, d_state, bias=False)
        self.s_D = nn.Sequential(nn.Linear(d_model, d_discr, bias=False), nn.Linear(d_discr, d_model, bias=False),)
        self.conv = nn.Conv1d(
            in_channels=d_model,
            out_channels=d_model,
            kernel_size=ker_size,
            padding=ker_size - 1,
            groups=d_model,
            bias=True,
        )
        self.A = nn.Parameter(torch.arange(1, d_state + 1, dtype=torch.float).repeat(d_model, 1))
        self.D = nn.Parameter(torch.ones(d_model, dtype=torch.float))
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def forward(self, seq, cache=None):
        b, l, d = seq.shape
        (prev_hid, prev_inp) = cache if cache is not None else (None, None)
        a, b = self.in_proj(seq).chunk(2, dim=-1)
        x = rearrange(a, 'b l d -> b d l')
        x = x if prev_inp is None else torch.cat((prev_inp, x), dim=-1)
        a = self.conv(x)[..., :l]
        a = rearrange(a, 'b d l -> b l d')
        a = silu(a)
        a, hid = self.ssm(a, prev_hid=prev_hid) 
        b = silu(b)
        out = a * b
        out =  self.out_proj(out)
        if cache:
            cache = (hid.squeeze(), x[..., 1:])   
        return out, cache
    
    def ssm(self, seq, prev_hid):
        A = -self.A
        D = +self.D
        B = self.s_B(seq)
        C = self.s_C(seq)
        s = softplus(D + self.s_D(seq))
        A_bar = einsum(torch.exp(A), s, 'd s,   b l d -> b l d s')
        B_bar = einsum(          B,  s, 'b l s, b l d -> b l d s')
        X_bar = einsum(B_bar, seq, 'b l d s, b l d -> b l d s')
        hid = self._hid_states(A_bar, X_bar, prev_hid=prev_hid)
        out = einsum(hid, C, 'b l d s, b l s -> b l d')
        out = out + D * seq
        return out, hid
    
    def _hid_states(self, A, X, prev_hid=None):
        b, l, d, s = A.shape
        A = rearrange(A, 'b l d s -> l b d s')
        X = rearrange(X, 'b l d s -> l b d s')
        if prev_hid is not None:
            return rearrange(A * prev_hid + X, 'l b d s -> b l d s')
        h = torch.zeros(b, d, s, device=self.device)
        return torch.stack([h := A_t * h + X_t for A_t, X_t in zip(A, X)], dim=1)

#### Mamba + jina

In [12]:
%%capture --no-stdout
result = []
ker_size = 4
num_layers = 1
for d_model in [64, 128, 256, 512]:
    print(f"d_model={d_model}, num_layers={num_layers}, ker_size={ker_size}")
    model_mamba = Mamba(model_name='jina', pooling=None,  num_layers = num_layers, d_input = 1024, d_model = d_model, num_classes=7, ker_size=ker_size).to(device)
    optimizer = optim.Adam(params = model_mamba.parameters(), lr = LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    trainer = ModelTrainer(model_mamba, train_dataloader, dev_meld_dataloader, dev_resd_dataloader, test_meld_dataloader, test_resd_dataloader, device, EPOCHS, ROUND_LOSS, ROUND_ACC, optimizer, loss_fn)
    trainer.train(PATH_TO_MODEL)
    checkpoint = torch.load(os.path.join(PATH_TO_MODEL, trainer._best_model_name))
    model_mamba.load_state_dict(checkpoint['model_state_dict'])
    metrics_dev_meld = evaluate_metrics(model_mamba, dev_meld_dataloader)
    metrics_dev_resd = evaluate_metrics(model_mamba, dev_resd_dataloader)
    print("Метрики на валидационной выборке MELD: ", metrics_dev_meld)
    print("Метрики на валидационной выборке RESD: ", metrics_dev_resd)
    metrics_test_meld = evaluate_metrics(model_mamba, test_meld_dataloader)
    metrics_test_resd = evaluate_metrics(model_mamba, test_resd_dataloader)
    print("Метрики на тестовой выборке MELD: ", metrics_test_meld)
    print("Метрики на тестовой выборке RESD: ", metrics_test_resd)
    result.append([{"d_model" : d_model, "num_layers": num_layers, "ker_size" : ker_size}, metrics_dev_meld, metrics_dev_resd, metrics_test_meld, metrics_test_resd, trainer._best_model_name])

d_model=64, num_layers=1, ker_size=4
Ранняя остановка на эпохе 24 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 40.1453592524256, 'war': 51.75834084761046, 'mf1': 39.115122572672384, 'wf1': 51.694737671216316}
Метрики на валидационной выборке RESD:  {'uar': 30.871058568354876, 'war': 31.044776119402982, 'mf1': 30.719674862246784, 'wf1': 31.292495874503068}
Метрики на тестовой выборке MELD:  {'uar': 36.56062782494112, 'war': 51.91570881226054, 'mf1': 34.95508193450415, 'wf1': 52.95587794935126}
Метрики на тестовой выборке RESD:  {'uar': 28.617272860693916, 'war': 28.92857142857143, 'mf1': 28.003793275614036, 'wf1': 28.358252039735817}
d_model=128, num_layers=1, ker_size=4
Ранняя остановка на эпохе 18 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 40.03732086702748, 'war': 48.69251577998197, 'mf1': 36.8872681279535, 'wf1': 49.79697284835936}
Метрики на валидационной выборке 

In [13]:
df = pd.DataFrame(result, columns=["параметры", "метрики dev meld", "метрики dev resd", "метрики test meld", "метрики test resd", "путь"])
df = pd.concat([df["параметры"].apply(pd.Series), df["метрики dev meld"].apply(pd.Series), df["метрики dev resd"].apply(pd.Series), df["метрики test meld"].apply(pd.Series), df["метрики test resd"].apply(pd.Series), df["путь"]], axis=1)
df.columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df.to_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_1_ker_size_4_d_model.csv"))

In [14]:
%%capture --no-stdout
result = []
ker_size = 4
num_layers = 2
for d_model in [64, 128, 256, 512]:
    print(f"d_model={d_model}, num_layers={num_layers}, ker_size={ker_size}")
    model_mamba = Mamba(model_name='jina', pooling=None,  num_layers = num_layers, d_input = 1024, d_model = d_model, num_classes=7, ker_size=ker_size).to(device)
    optimizer = optim.Adam(params = model_mamba.parameters(), lr = LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    trainer = ModelTrainer(model_mamba, train_dataloader, dev_meld_dataloader, dev_resd_dataloader, test_meld_dataloader, test_resd_dataloader, device, EPOCHS, ROUND_LOSS, ROUND_ACC, optimizer, loss_fn)
    trainer.train(PATH_TO_MODEL)
    checkpoint = torch.load(os.path.join(PATH_TO_MODEL, trainer._best_model_name))
    model_mamba.load_state_dict(checkpoint['model_state_dict'])
    metrics_dev_meld = evaluate_metrics(model_mamba, dev_meld_dataloader)
    metrics_dev_resd = evaluate_metrics(model_mamba, dev_resd_dataloader)
    print("Метрики на валидационной выборке MELD: ", metrics_dev_meld)
    print("Метрики на валидационной выборке RESD: ", metrics_dev_resd)
    metrics_test_meld = evaluate_metrics(model_mamba, test_meld_dataloader)
    metrics_test_resd = evaluate_metrics(model_mamba, test_resd_dataloader)
    print("Метрики на тестовой выборке MELD: ", metrics_test_meld)
    print("Метрики на тестовой выборке RESD: ", metrics_test_resd)
    result.append([{"d_model" : d_model, "num_layers": num_layers, "ker_size" : ker_size}, metrics_dev_meld, metrics_dev_resd, metrics_test_meld, metrics_test_resd, trainer._best_model_name])

d_model=64, num_layers=2, ker_size=4
Ранняя остановка на эпохе 18 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 41.76370101544592, 'war': 51.12714156898106, 'mf1': 40.16349389364534, 'wf1': 52.202720863128135}
Метрики на валидационной выборке RESD:  {'uar': 31.820101953825002, 'war': 32.537313432835816, 'mf1': 31.139894370659267, 'wf1': 31.819726057646424}
Метрики на тестовой выборке MELD:  {'uar': 40.59738150619937, 'war': 53.63984674329502, 'mf1': 38.520091112382204, 'wf1': 55.188357863518235}
Метрики на тестовой выборке RESD:  {'uar': 30.718786113522956, 'war': 31.428571428571427, 'mf1': 30.135800630050692, 'wf1': 30.647716371474626}
d_model=128, num_layers=2, ker_size=4
Ранняя остановка на эпохе 17 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 42.189028169429285, 'war': 49.77457168620379, 'mf1': 39.163589067629296, 'wf1': 51.44236558208918}
Метрики на валидационной вы

In [15]:
df = pd.DataFrame(result, columns=["параметры", "метрики dev meld", "метрики dev resd", "метрики test meld", "метрики test resd", "путь"])
df = pd.concat([df["параметры"].apply(pd.Series), df["метрики dev meld"].apply(pd.Series), df["метрики dev resd"].apply(pd.Series), df["метрики test meld"].apply(pd.Series), df["метрики test resd"].apply(pd.Series), df["путь"]], axis=1)
df.columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df.to_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_2_ker_size_4_d_model.csv"))

In [16]:
%%capture --no-stdout
result = []
ker_size = 4
num_layers = 3
for d_model in [64, 128, 256, 512]:
    print(f"d_model={d_model}, num_layers={num_layers}, ker_size={ker_size}")
    model_mamba = Mamba(model_name='jina', pooling=None,  num_layers = num_layers, d_input = 1024, d_model = d_model, num_classes=7, ker_size=ker_size).to(device)
    optimizer = optim.Adam(params = model_mamba.parameters(), lr = LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    trainer = ModelTrainer(model_mamba, train_dataloader, dev_meld_dataloader, dev_resd_dataloader, test_meld_dataloader, test_resd_dataloader, device, EPOCHS, ROUND_LOSS, ROUND_ACC, optimizer, loss_fn)
    trainer.train(PATH_TO_MODEL)
    checkpoint = torch.load(os.path.join(PATH_TO_MODEL, trainer._best_model_name))
    model_mamba.load_state_dict(checkpoint['model_state_dict'])
    metrics_dev_meld = evaluate_metrics(model_mamba, dev_meld_dataloader)
    metrics_dev_resd = evaluate_metrics(model_mamba, dev_resd_dataloader)
    print("Метрики на валидационной выборке MELD: ", metrics_dev_meld)
    print("Метрики на валидационной выборке RESD: ", metrics_dev_resd)
    metrics_test_meld = evaluate_metrics(model_mamba, test_meld_dataloader)
    metrics_test_resd = evaluate_metrics(model_mamba, test_resd_dataloader)
    print("Метрики на тестовой выборке MELD: ", metrics_test_meld)
    print("Метрики на тестовой выборке RESD: ", metrics_test_resd)
    result.append([{"d_model" : d_model, "num_layers": num_layers, "ker_size" : ker_size}, metrics_dev_meld, metrics_dev_resd, metrics_test_meld, metrics_test_resd, trainer._best_model_name])

d_model=64, num_layers=3, ker_size=4
Ранняя остановка на эпохе 19 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 39.24013517936903, 'war': 52.209197475202885, 'mf1': 38.746034687077724, 'wf1': 52.5657535793767}
Метрики на валидационной выборке RESD:  {'uar': 31.846686500687234, 'war': 32.23880597014925, 'mf1': 31.61177058355803, 'wf1': 32.109376801299476}
Метрики на тестовой выборке MELD:  {'uar': 38.59887085742035, 'war': 53.14176245210728, 'mf1': 36.90749865066169, 'wf1': 54.51596823231611}
Метрики на тестовой выборке RESD:  {'uar': 27.648410543147385, 'war': 28.214285714285715, 'mf1': 27.719212866504993, 'wf1': 28.497896823782593}
d_model=128, num_layers=3, ker_size=4
Ранняя остановка на эпохе 16 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 41.516372530320176, 'war': 50.85662759242561, 'mf1': 38.258880721913066, 'wf1': 51.02863024719397}
Метрики на валидационной выборк

In [17]:
df = pd.DataFrame(result, columns=["параметры", "метрики dev meld", "метрики dev resd", "метрики test meld", "метрики test resd", "путь"])
df = pd.concat([df["параметры"].apply(pd.Series), df["метрики dev meld"].apply(pd.Series), df["метрики dev resd"].apply(pd.Series), df["метрики test meld"].apply(pd.Series), df["метрики test resd"].apply(pd.Series), df["путь"]], axis=1)
df.columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df.to_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_3_ker_size_4_d_model.csv"))

In [18]:
columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df = pd.concat([pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_1_ker_size_4_d_model.csv"), index_col=0), pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_2_ker_size_4_d_model.csv"), index_col=0), pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_3_ker_size_4_d_model.csv"), index_col=0)])
df.columns=columns

In [20]:
df['average_dev_meld'] = (df['uar_dev_meld'] + df['war_dev_meld'] + df['mf1_dev_meld'] + df['wf1_dev_meld']) / 4.0
df['average_dev_resd'] = (df['uar_dev_resd'] + df['war_dev_resd'] + df['mf1_dev_resd'] + df['wf1_dev_resd']) / 4.0
df['average_test_meld'] = (df['uar_test_meld'] + df['war_test_meld'] + df['mf1_test_meld'] + df['wf1_test_meld']) / 4.0
df['average_test_resd'] = (df['uar_test_resd'] + df['war_test_resd'] + df['mf1_test_resd'] + df['wf1_test_resd']) / 4.0

In [33]:
df.sort_values(['average_test_resd', 'average_test_meld'] , ascending=False)

,d_model,num_layers,ker_size,uar_dev_meld,war_dev_meld,mf1_dev_meld,wf1_dev_meld,uar_dev_resd,war_dev_resd,mf1_dev_resd,...,wf1_test_meld,uar_test_resd,war_test_resd,mf1_test_resd,wf1_test_resd,путь,average_dev_meld,average_dev_resd,average_test_meld,average_test_resd
3,512,2,4,39.408059,54.914337,40.489086,53.028490,35.569400,35.820896,35.508898,...,55.268566,37.200911,37.500000,37.175183,37.356274,Mamba_jina_25_41.19_checkpoint.pth,46.959993,35.694853,46.159247,37.308092
2,256,2,4,36.618373,49.413886,36.077703,49.244611,35.923668,36.417910,35.455143,...,54.136911,36.247523,36.428571,36.692780,36.864663,Mamba_jina_12_38.87_checkpoint.pth,42.838643,35.923612,45.331295,36.558384
3,512,3,4,42.546295,53.471596,41.691935,53.289745,34.613148,34.925373,34.855382,...,54.887694,34.519766,35.000000,34.314357,34.668531,Mamba_jina_5_41.4_checkpoint.pth,47.749893,34.885006,46.653132,34.625663
1,128,3,4,41.516373,50.856628,38.258881,51.028630,32.734070,32.835821,32.228419,...,53.305432,32.917755,33.214286,32.042812,32.539638,Mamba_jina_6_38.54_checkpoint.pth,45.415128,32.674703,44.853900,32.678623
1,128,1,4,40.037321,48.692516,36.887268,49.796973,33.293573,33.432836,32.887290,...,49.912187,32.417530,32.857143,31.723123,32.311382,Mamba_jina_8_37.64_checkpoint.pth,43.853519,33.199220,42.792389,32.327295
2,256,3,4,41.209792,47.971145,38.082530,49.577856,36.362515,37.014925,35.983411,...,48.737094,32.215697,32.500000,30.971970,31.168165,Mamba_jina_6_40.1_checkpoint.pth,44.210331,36.480246,42.070172,31.713958
3,512,1,4,38.897907,51.668170,36.979896,50.770400,33.072608,32.835821,32.881661,...,54.539324,31.514808,31.785714,31.523172,31.974245,Mamba_jina_13_38.3_checkpoint.pth,44.579093,32.974173,45.527194,31.699485
2,256,1,4,38.140976,50.135257,37.543032,49.555923,33.650738,33.134328,32.920493,...,53.716002,31.009193,31.071429,31.019941,31.411611,Mamba_jina_9_38.34_checkpoint.pth,43.843797,33.199782,45.196504,31.128043
0,64,2,4,41.763701,51.127142,40.163494,52.202721,31.820102,32.537313,31.139894,...,55.188358,30.718786,31.428571,30.135801,30.647716,Mamba_jina_8_38.23_checkpoint.pth,46.314264,31.829259,46.986419,30.732719
0,64,1,4,40.145359,51.758341,39.115123,51.694738,30.871059,31.044776,30.719675,...,52.955878,28.617273,28.928571,28.003793,28.358252,Mamba_jina_14_38.13_checkpoint.pth,45.678390,30.982001,44.096824,28.476972


In [22]:
model_mamba_best = Mamba(num_layers = 2, d_input = 1024, d_model = 512, num_classes=7, model_name='jina', pooling=None).to(device)
checkpoint = torch.load(os.path.join(PATH_TO_MODEL, "Mamba_jina_25_41.19_checkpoint.pth"))
model_mamba_best.load_state_dict(checkpoint['model_state_dict'])

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

<All keys matched successfully>

In [23]:
evaluate_metrics(model_mamba_best, test_meld_dataloader)

{'uar': 36.168594927705044,
 'war': 56.43678160919541,
 'mf1': 36.763044570154726,
 'wf1': 55.26856554959636}

In [24]:
evaluate_metrics(model_mamba_best, test_resd_dataloader)

{'uar': 37.20091075354233,
 'war': 37.5,
 'mf1': 37.175183397267936,
 'wf1': 37.3562736499167}

In [13]:
%%capture --no-stdout
result = []
for ker_size in [2, 8]:
    for (d_model, num_layers) in [(512, 2), (256, 2)]:
        print(f"d_model={d_model}, num_layers={num_layers}, ker_size={ker_size}")
        model_mamba = Mamba(model_name='jina', pooling=None,  num_layers = num_layers, d_input = 1024, d_model = d_model, num_classes=7, ker_size=ker_size).to(device)
        optimizer = optim.Adam(params = model_mamba.parameters(), lr = LEARNING_RATE)
        loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        trainer = ModelTrainer(model_mamba, train_dataloader, dev_meld_dataloader, dev_resd_dataloader, test_meld_dataloader, test_resd_dataloader, device, EPOCHS, ROUND_LOSS, ROUND_ACC, optimizer, loss_fn)
        trainer.train(PATH_TO_MODEL)
        checkpoint = torch.load(os.path.join(PATH_TO_MODEL, trainer._best_model_name))
        model_mamba.load_state_dict(checkpoint['model_state_dict'])
        metrics_dev_meld = evaluate_metrics(model_mamba, dev_meld_dataloader)
        metrics_dev_resd = evaluate_metrics(model_mamba, dev_resd_dataloader)
        print("Метрики на валидационной выборке MELD: ", metrics_dev_meld)
        print("Метрики на валидационной выборке RESD: ", metrics_dev_resd)
        metrics_test_meld = evaluate_metrics(model_mamba, test_meld_dataloader)
        metrics_test_resd = evaluate_metrics(model_mamba, test_resd_dataloader)
        print("Метрики на тестовой выборке MELD: ", metrics_test_meld)
        print("Метрики на тестовой выборке RESD: ", metrics_test_resd)
        result.append([{"d_model" : d_model, "num_layers": num_layers, "ker_size" : ker_size}, metrics_dev_meld, metrics_dev_resd, metrics_test_meld, metrics_test_resd, trainer._best_model_name])

d_model=512, num_layers=2, ker_size=2
Ранняя остановка на эпохе 17 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 41.62485082227551, 'war': 49.95491433724076, 'mf1': 39.387680121808145, 'wf1': 50.70561248038333}
Метрики на валидационной выборке RESD:  {'uar': 39.84149988497814, 'war': 39.701492537313435, 'mf1': 38.8124310751948, 'wf1': 39.22532756878456}
Метрики на тестовой выборке MELD:  {'uar': 38.41799053621324, 'war': 49.88505747126437, 'mf1': 35.75268408394602, 'wf1': 51.53520604639389}
Метрики на тестовой выборке RESD:  {'uar': 32.353272616430516, 'war': 33.214285714285715, 'mf1': 31.87809939681777, 'wf1': 32.663925575095575}
d_model=256, num_layers=2, ker_size=2
Ранняя остановка на эпохе 20 из-за отсутствия улучшения точности на тестовой выборке
Метрики на валидационной выборке MELD:  {'uar': 38.15311220636128, 'war': 51.307484220018026, 'mf1': 38.127662108208604, 'wf1': 51.21426943011797}
Метрики на валидационной выборке 

In [14]:
df = pd.DataFrame(result, columns=["параметры", "метрики dev meld", "метрики dev resd", "метрики test meld", "метрики test resd", "путь"])
df = pd.concat([df["параметры"].apply(pd.Series), df["метрики dev meld"].apply(pd.Series), df["метрики dev resd"].apply(pd.Series), df["метрики test meld"].apply(pd.Series), df["метрики test resd"].apply(pd.Series), df["путь"]], axis=1)
df.columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df.to_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_ker_size_d_model.csv"))

In [16]:
columns = ["d_model", "num_layers", "ker_size", "uar_dev_meld", "war_dev_meld", "mf1_dev_meld", "wf1_dev_meld", "uar_dev_resd", "war_dev_resd", "mf1_dev_resd", "wf1_dev_resd", "uar_test_meld", "war_test_meld", "mf1_test_meld", "wf1_test_meld", "uar_test_resd", "war_test_resd", "mf1_test_resd", "wf1_test_resd", "путь"]
df = pd.concat([pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_1_ker_size_4_d_model.csv"), index_col=0), pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_2_ker_size_4_d_model.csv"), index_col=0), pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_3_ker_size_4_d_model.csv"), index_col=0), pd.read_csv(os.path.join(PATH_TO_MODEL, "result_num_layers_ker_size_d_model.csv"), index_col=0)])
df.columns=columns

In [17]:
df['average_dev_meld'] = (df['uar_dev_meld'] + df['war_dev_meld'] + df['mf1_dev_meld'] + df['wf1_dev_meld']) / 4.0
df['average_dev_resd'] = (df['uar_dev_resd'] + df['war_dev_resd'] + df['mf1_dev_resd'] + df['wf1_dev_resd']) / 4.0
df['average_test_meld'] = (df['uar_test_meld'] + df['war_test_meld'] + df['mf1_test_meld'] + df['wf1_test_meld']) / 4.0
df['average_test_resd'] = (df['uar_test_resd'] + df['war_test_resd'] + df['mf1_test_resd'] + df['wf1_test_resd']) / 4.0

In [19]:
df.sort_values(['average_test_resd', 'average_test_meld'] , ascending=False)

,d_model,num_layers,ker_size,uar_dev_meld,war_dev_meld,mf1_dev_meld,wf1_dev_meld,uar_dev_resd,war_dev_resd,mf1_dev_resd,...,wf1_test_meld,uar_test_resd,war_test_resd,mf1_test_resd,wf1_test_resd,путь,average_dev_meld,average_dev_resd,average_test_meld,average_test_resd
3,512,2,4,39.408059,54.914337,40.489086,53.028490,35.569400,35.820896,35.508898,...,55.268566,37.200911,37.500000,37.175183,37.356274,Mamba_jina_25_41.19_checkpoint.pth,46.959993,35.694853,46.159247,37.308092
2,256,2,4,36.618373,49.413886,36.077703,49.244611,35.923668,36.417910,35.455143,...,54.136911,36.247523,36.428571,36.692780,36.864663,Mamba_jina_12_38.87_checkpoint.pth,42.838643,35.923612,45.331295,36.558384
3,512,3,4,42.546295,53.471596,41.691935,53.289745,34.613148,34.925373,34.855382,...,54.887694,34.519766,35.000000,34.314357,34.668531,Mamba_jina_5_41.4_checkpoint.pth,47.749893,34.885006,46.653132,34.625663
1,256,2,2,38.153112,51.307484,38.127662,51.214269,36.164114,35.820896,35.745036,...,53.932402,33.600899,33.928571,33.197634,33.489637,Mamba_jina_10_40.53_checkpoint.pth,44.700632,35.831813,45.047066,33.554185
1,128,3,4,41.516373,50.856628,38.258881,51.028630,32.734070,32.835821,32.228419,...,53.305432,32.917755,33.214286,32.042812,32.539638,Mamba_jina_6_38.54_checkpoint.pth,45.415128,32.674703,44.853900,32.678623
0,512,2,2,41.624851,49.954914,39.387680,50.705612,39.841500,39.701493,38.812431,...,51.535206,32.353273,33.214286,31.878099,32.663926,Mamba_jina_7_41.92_checkpoint.pth,45.418264,39.395188,43.897735,32.527396
2,512,2,8,42.243621,52.479711,39.895981,52.980979,35.506404,34.626866,34.122410,...,54.837972,33.072658,33.571429,31.392407,31.956568,Mamba_jina_6_40.1_checkpoint.pth,46.900073,34.600171,46.167395,32.498265
1,128,1,4,40.037321,48.692516,36.887268,49.796973,33.293573,33.432836,32.887290,...,49.912187,32.417530,32.857143,31.723123,32.311382,Mamba_jina_8_37.64_checkpoint.pth,43.853519,33.199220,42.792389,32.327295
2,256,3,4,41.209792,47.971145,38.082530,49.577856,36.362515,37.014925,35.983411,...,48.737094,32.215697,32.500000,30.971970,31.168165,Mamba_jina_6_40.1_checkpoint.pth,44.210331,36.480246,42.070172,31.713958
3,512,1,4,38.897907,51.668170,36.979896,50.770400,33.072608,32.835821,32.881661,...,54.539324,31.514808,31.785714,31.523172,31.974245,Mamba_jina_13_38.3_checkpoint.pth,44.579093,32.974173,45.527194,31.699485
